In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import open3d as o3d

In [ ]:
def load_and_fix_data(filename):
    print(f"Reading {filename}...")
    try:
        # 1. Load the flat text file
        # This reads the long list of numbers
        raw_data = np.loadtxt(filename)

        # 2. Reshape the data
        # The '-1' tells numpy to automatically calculate the number of rows.
        # The '4' is because CARLA lidar data is (x, y, z, intensity).
        points = raw_data.reshape(-1, 4)

        print(f"Success! Restored point cloud with shape: {points.shape}")
        return points

    except Exception as e:
        print(f"Error loading file: {e}")
        return None

def visualize_bev(points, lidar_range=50, width=512, height=512):
    if points is None: return

    # --- Your Original Projection Logic ---
    pixels_per_meter = width / (2 * lidar_range)

    # Extract columns
    lidar_x = points[:, 0]
    lidar_y = points[:, 1]
    lidar_z = points[:, 2]

    # Filter points outside the range
    mask = (np.abs(lidar_x) <= lidar_range) & (np.abs(lidar_y) <= lidar_range)
    lidar_x = lidar_x[mask]
    lidar_y = lidar_y[mask]
    lidar_z = lidar_z[mask]

    # Map to image coordinates
    # In CARLA: X=Forward, Y=Right
    # Image: u (width) maps to Y, v (height) maps to -X
    u = (width / 2) + lidar_y * pixels_per_meter
    v = (height / 2) - lidar_x * pixels_per_meter

    u = np.clip(u, 0, width - 1).astype(int)
    v = np.clip(v, 0, height - 1).astype(int)

    # Draw Image
    lidar_img = np.zeros((height, width, 3), dtype=np.uint8)

    # Blue channel = fixed, Green channel = height (z)
    lidar_img[v, u, 2] = 255
    lidar_img[v, u, 1] = np.clip((lidar_z + 2.0) * 50, 0, 255).astype(np.uint8)

    # --- Plotting ---
    plt.figure(figsize=(8, 8))
    plt.imshow(lidar_img)
    plt.title(f"Bird's Eye View (Points: {len(lidar_x)})")
    plt.axis('off') # Hide axes for cleaner look
    plt.show()

In [ ]:
# Load the data
filename = "lidar_snapshot.txt"
points = load_and_fix_data(filename)
# Run 2D Vis
if points is not None:
    visualize_bev(points)